# **Introduction to CUDA Python with Numba**

## **CONCEPT 1: What is Numba?**

Numba is a **just-in-time, type-specializing, function compiler** for accelerating **numerically-focused** Python. Let's break down those terms:

- **function compiler:** Numba compiles Python functions, not entire applications, and not parts of functions. Numba does not replace your Python interpreter, but is just another Python module that can turn a function into a (usually) faster function.
- **type-specializing:** Numba speeds up your function by generating a specialized implementation for the specific data types you are using. Python functions are designed to operate on generic data types, which makes them very flexible, but also very slow. In practice, you only will call a function with a small number of argument types, so Numba will generate a fast implementation for each set of types.
- **just-in-time:** Numba translates functions when they are first called. This ensures the compiler knows what argument types you will be using. This also allows Numba to be used interactively in a Jupyter notebook just as easily as a traditional application
- **numerically-focused:** Currently, Numba is focused on numerical data types, like int, float, and complex. There is very limited string processing support, and many string use cases are not going to work well on the GPU. To get best results with Numba, you will likely be using NumPy arrays.

The most common way to use Numba is through its collection of decorators that can be applied to your functions to instruct Numba to compile them. When a call is made to a Numba decorated function it is compiled to machine code “just-in-time” for execution and all or part of your code can subsequently run at native machine code speed!

Numba supports CUDA GPU programming by directly compiling a restricted subset of Python code into CUDA kernels and device functions following the CUDA execution model. Kernels written in Numba appear to have direct access to NumPy arrays. NumPy arrays are transferred between the CPU and the GPU automatically.

[Documentation](https://numba.readthedocs.io/en/stable/cuda/index.html)

#### **Terminology**
* **host**: the CPU
* **device**: the GPU
* **host memory**: the system main memory
* **device memory**: onboard memory on a GPU card
* **kernels**: a GPU function launched by the host and executed on the device
* **device function**: a GPU function executed on the device which can only be called from the device (i.e. from a kernel or another device function)

Most CUDA programming facilities exposed by Numba map directly to the CUDA C language offered by NVIDIA. Therefore, it is recommended you read the official [CUDA C programming guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/).

## **CONCEPT 2: How does Numba work?**

Numba reads the Python bytecode for a decorated function and combines this with information about the types of the input arguments to the function. It analyzes and optimizes your code, and finally uses the LLVM compiler library to generate a machine code version of your function, tailored to your CPU capabilities. This compiled version is then used every time your function is called.

![Numba flowcart](../numba_flowcart.png)

### **Other things of interest:**

Numba has quite a few decorators:
- `@jit` - compile the decorated function on-the-fly to produce efficient machine code. [Docs are here](https://numba.pydata.org/numba-doc/latest/reference/jit-compilation.html)
- `@njit` - this is an alias for `@jit(nopython=True)` as it is so commonly used!
- `@vectorize` - produces NumPy `ufunc` s (with all the `ufunc` methods supported). [Docs are here](https://numba.pydata.org/numba-doc/latest/user/vectorize.html#vectorize).
- `@guvectorize` - produces NumPy generalized `ufunc` s. [Docs are here](https://numba.pydata.org/numba-doc/latest/user/vectorize.html#guvectorize).
- `@stencil` - declare a function as a kernel for a stencil like operation. [Docs are here](https://numba.pydata.org/numba-doc/latest/user/stencil.html#numba-stencil).
- `@jitclass` - for jit aware classes. [Docs are here](https://numba.pydata.org/numba-doc/latest/user/jitclass.html#jitclass).
- `@cfunc` - declare a function for use as a native call back (to be called from C/C++ etc). [Docs are here](https://numba.pydata.org/numba-doc/latest/user/cfunc.html#cfunc).
- `@overload` - register your own implementation of a function for use in nopython mode, e.g. `@overload(scipy.special.j0)`. [Docs are here](https://numba.pydata.org/numba-doc/latest/extending/high-level.html#high-level-extending).





# **Tutorial: Numba Basics**

For this tutorial, we will learn how to write a Numba function and compile it for the CPU, how to create a universal Numpy functions and how to write device functions.

## **Compiling Python code with @jit**

Numba provides several utilities for code generation, but its central feature is the numba.jit() decorator with an explicit signature. Using Numba decorator @jit, which creates a normal function for execution on the CPU.

### Step 1: Import libraries

In [ ]:
import numpy as np
import math
import random
from decimal import Decimal

import numba
from numba import jit
from numba import vectorize 
from numba import guvectorize
from numba import cuda

In [ ]:
# Checking Numba version
print(numba.__version__)

### Step 2: Compiling functions on the CPU

We will write the first Numba function and compile it for the CPU. The Numba compiler is typically enabled by applying a decorator to a Python function. Decorators are functions that transform Python functions. Here we will use the CPU compilation decorator:

 Let’s start by peeking at the numba.jit string-doc:

In [ ]:
print(numba.jit.__doc__)

The recommended way to use the @jit decorator is to let Numba decide when and how to optimize. Let's make a compiled version of a function. In this mode, compilation will be deferred until the first function execution. Numba will infer the argument types at call time, and generate optimized code based on this information.

In [ ]:
nsamples = 2000
# JIT compile a python function conforming to the CUDA-Python specification. To define a CUDA kernel:
@jit
def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

### Step 3: Benchmarking

Measuring the performance of your new code is an important part. Let's see if we actuall speed up between the compiled and uncompiled versions. The easiest way to do is in the Jupyter notebook is to use **%timeit** function.
 **.py_func** attribute that can be used to access the original uncompiled Python function.

In [ ]:
%timeit monte_carlo_pi.py_func(nsamples)

In [ ]:
%timeit monte_carlo_pi(nsamples)

The Numba-compiled version had a significant speed-up with this function!

### **Signature specifications**

In order to generate fast code, the compiler needs type information for the code. This allows a direct mapping from the Python operations to the appropriate machine instruction without any type check/dispatch mechanism. 

Explicit `@jit` signatures can use a number of types. Here are some common ones:

- `void` is the return type of functions returning nothing (which actually return `None` when called from Python)
- `intp` and `uintp` are pointer-sized integers (signed and unsigned, respectively)
- `intc` and `uintc` are equivalent to C `int` and `unsigned int` integer types
- `int8`, `uint8`, `int16`, `uint16`, `int32`, `uint32`, `int64`, `uint64` are fixed-width integers of the corresponding bit width (signed and unsigned)
- `float32` and `float64` are single- and double-precision floating-point numbers, respectively
- `complex64` and `complex128` are single- and double-precision complex numbers, respectively
- array types can be specified by indexing any numeric type, e.g. `float32[:]` for a one-dimensional single-precision array or `int8[:,:]` for a two-dimensional array of 8-bit integers.



### Step 4: Compilation options

The Numba @jit decorator fundamentally operates in two compilation modes:
- `nopython mode`
- `object mode`

The behaviour of the **nopython** compilation mode is to essentially compile the decorated function so that it will run entirely without the involvement of the Python interpreter. This is the recommended and best-practice way to use the Numba jit decorator as it leads to the best performance.

Should the compilation in **nopython mode** fail, Numba can compile using **object mode**. This is a fall back mode for the @jit decorator if **nopython=True** is not set. In this mode Numba will identify loops that it can compile and compile those into functions that run in machine code, and it will run the rest of the code in the interpreter. For best performance avoid using this mode!

Numba cannot compile all Python code. Some functions don't have a Numba-translation, and some kinds of Python types can't be efficiently compiled at all (yet). For example, Numba does not support dictionaries (as of this tutorial):

In [ ]:
@jit
def cannot_compile(x):
    return x['key']

cannot_compile(dict(key='value'))

By default, Numba will fall back to a mode, called "object mode," which does not do type-specialization. Object mode exists to enable other Numba functionality, but in many cases, you want Numba to tell you if type inference fails. You can force "nopython mode" (the other compilation mode) by passing arguments to the decorator:

Let's see nopython mode. The nopython mode will generate the best performance, but has limitations.

In [ ]:
#function without error
@jit("void(f4[:])",nopython=True)
def squared(a):
    squared_val = a*a


Types that can’t be inferred by the compiler in the nopython mode and it will generate an error.

In [ ]:
#function that contains a variable whose type can’t be inferred by the compiler
#nopython mode set to True

@jit("void(f4[:])",nopython=True)
def squared(a):
    squared_val = a*a
    val = Decimal(100)

If we don’t specify anything like in the function below, where nopython mode is not set to True, the compilation is falling back to object mode and produces a warning but not an error.

In [ ]:
# function that contains a variable whose type can’t be inferred by the compiler
# nopython mode not set to True
# compilation is falling back to object mode

@jit("void(f4[:])")
def squared(a):
    squared_val = a*a
    val = Decimal(100)

### Step 4: Type inference

The objective of type inference is assigning a type to every single value in the function. The type of a value can either be:

* Implicit, in the case of providing an object that will provide its type. For e.g. in literals.
* Explicit, in the case of the programmer explicitly writing the type of a given value. For e.g. when a signature is given to numba.jit. That signature explicitly types the arguments.
* Inferred, when the type is deduced from an operation and the types of its operands. For e.g. inferring that the type of a + b, when a and b are of type int is going to be an int
Type inference is the process by which all the types that are neither implicit nor explicit are deduced.

In [ ]:
# Let’s illustrate how type inference works with numba.jit. 
# We can see the result of type inference by using the .inspect_types() method, which prints an annotated version of the source code.

monte_carlo_pi.inspect_types()

Note that Numba's type names tend to mirror the NumPy type names, so a Python float is a float64 (also called "double precision" in other languages). Taking a look at the data types can sometimes be important in GPU code because the performance of float32 and float64 computations will be very different on CUDA devices. An accidental upcast can dramatically slow down a function.

## **Creating Numpy universal functions**

#### **CUDA Basics**

There are two basic approaches to GPU programming in Numba:

- ufuncs/gufuncs
- CUDA Python kernels. [See documentation here](https://numba.pydata.org/numba-doc/latest/cuda/kernels.html)

We will not go into the CUDA hardware too much in this section, but the most important thing to remember is that the hardware is designed for data parallelism. Maximum throughput is achieved when you are computing the same operations on many different elements at once.

Universal functions are naturally data parallel, so we will begin with them.

#### **Universal Functions** 

Numba supports generating two types of universal functions:

- Those which operate on scalars, these are “universal functions” or ufuncs (@vectorize below).
- Those which operate on higher dimensional arrays and scalars, these are “generalized universal functions” or gufuncs (@guvectorize below).

#### **Universal functions**

Universal functions(ufuncs) are functions that broadcast an elementwise operation across input arrays of varying numbers of dimensions. Most NumPy functions are ufuncs, and Numba makes it easy to compile custom ufuncs using the @vectorize decorator.

#### Step 1: 

The NumPy documentation has a much more extensive discussion of ufuncs:

https://docs.scipy.org/doc/numpy/reference/ufuncs.html

In [ ]:
# Applying sqrt() method in Numpy
def cpu_sqrt(x):
  return math.sqrt(x)

In [ ]:
# Making ufuncs for the CPU
@vectorize
def cpu_numba_sqrt(x):
  return math.sqrt(x)

In [ ]:
# Making a ufunc for the GPU
# Generating a ufunc that uses CUDA requires giving an explicit type signature and setting the target attribute
@vectorize(['float32(float32)'], target='cuda')
def gpu_numba_sqrt(x):
    return math.sqrt(x)

In [ ]:
%timeit np.sqrt(25) # NumPy on CPU

In [ ]:
%timeit cpu_sqrt(25)  # NumPy on CPU

In [ ]:
%timeit cpu_numba_sqrt(25) # Numba on CPU

In [ ]:
%timeit gpu_numba_sqrt(25) # Numba on GPU

Why did we see an increase in the processing time?

An overhead is introduced by Numba to each function call that is larger than the function call overhead of Python itself. 

Functions that are quick to compute are affected by this.

Since the function here is too simple to run on the GPU, we experience longer processing time due to data transfer operations taking place.

* input data is transferred to the GPU memory
* the square root is calculated on the GPU
* the resulting value is sent back to the host system

#### **Generalized ufuncs on the GPU**

In generalized ufuncs (gufuncs), the calculation can deal with a sub-array of the input array, and return an array of different dimensions.

In [ ]:
#calculating the sum of elements in each row of the array
@guvectorize(['(float32[:], float32[:])'],
             '(n)->()',                
             target='cuda')
def calc_sum(a, out):
    sum = 0
    for val in a: 
        sum += val
    out[0] = sum

In [ ]:
a = np.arange(50).reshape(10, 5).astype(np.float32)
a

In [ ]:
calc_sum(a)

## **CUDA Device Functions**

Uptil now we were compiling all our code in single functions and ufuncs are great, but you should not have to cram all of your logic into a single function body. You can also create normal functions that are only called from other functions running on the GPU. These are called **device** functions.(These are similar to CUDA C functions defined with __device__.)

Device functions are created with the numba.cuda.jit decorator:

`add_values:` device function to sum up the elements of the array using the numba.cuda.jit decorator

`calc_sum:` gufunc to make use of the add_values device function

`calc_avg:` gufunc to calculate the average

In [ ]:
# define a device function
@cuda.jit(device=True)
def add_values(a): 
  sum = 0
  for val in a: 
    sum += val
  return sum

# define a gufunc that calls our device function
@guvectorize(['(float32[:], float32[:])'],
             '(n)->()',                
             target='cuda')
def calc_sum(a, out):
    out[0] = add_values(a)

@guvectorize(['(float32[:], float32[:])'],
             '(n)->()',                
             target='cuda')
def calc_avg(a, out):
    out[0] = add_values(a)/len(a)

In [ ]:
calc_sum(a)

In [ ]:
calc_avg(a)

# **Exercise: Numba Basics**

The function below is a naive sum function that sums all the elements of a given array. Let’s get a numba version of this code running.

In [ ]:
#Write your own code

##### **As a function call**

Compile **summation_array()** function using numba.jit without providing a type-signature for the function.

In [ ]:
# Write your own code

##### **As a decorater**

In [ ]:
# Write your own code

##### **Benchmarking**

Compare NumPy and Numba using %timeit

In [ ]:
# Write your own code


##### **Question: When does Numba compile codes?**

Write your answer here.

# **Tutorial: Memory Management**

In order to reduce the impact of host-to-device/device-to-host bandwidth, it is best to copy data to the GPU explicitly and leave it there to amortize the cost over multiple function calls. In addition, allocating device memory can be relatively slow, so allocating GPU arrays once and refilling them with data from the host can also be a performance improvement.



In [ ]:
@vectorize(['float32(float32, float32)'], target='cuda')
def add_ufunc(x, y):
    return x + y

In [ ]:
n = 100000
x = np.arange(n).astype(np.float32)
y = 2 * x

In [ ]:
%timeit add_ufunc(x, y)  # Baseline performance with host arrays

The numba.cuda module includes a function that will copy host data to the GPU and return a CUDA device array:

In [ ]:
from numba import cuda

x_device = cuda.to_device(x)
y_device = cuda.to_device(y)

print(x_device)
print(x_device.shape)
print(x_device.dtype)

Device arrays can be passed to CUDA functions just like NumPy arrays, but without the copy overhead:

In [ ]:

%timeit add_ufunc(x_device, y_device)

That's a big performance improvement already, but we are still allocating a device array for the output of the ufunc and copying it back to the host. We can create the output buffer with the numba.cuda.device_array() function:

In [ ]:
out_device = cuda.device_array(shape=(n,), dtype=np.float32)  # does not initialize the contents, like np.empty()

And then we can use a special out keyword argument to the ufunc to specify the output buffer:

In [ ]:
%timeit add_ufunc(x_device, y_device, out=out_device)

Now that we have removed the device allocation and copy steps, the computation runs much faster than before. When we want to bring the device array back to the host memory, we can use the copy_to_host() method:

In [ ]:
out_host = out_device.copy_to_host()
print(out_host[:10])

# **Tutorial: Writing CUDA kernels**

Ufuncs and generalized ufuncs are the easiest way in Numba to use the GPU, and present an abstraction that requires minimal understanding of the CUDA programming model. However, you can not write all functions as ufuncs so you write a CUDA kernel.

It's highly recommend to read Chapters 1 and 2 of the CUDA C Programming Guide to understand how to write CUDA kernels with Numba.
- Introduction: http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#introduction
- Programming Model: http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#programming-model

![[cuda flowchart](https://camo.githubusercontent.com/87204582bdb0422a08cef47cd0bafcfd860cd522/687474703a2f2f646f63732e6e76696469612e636f6d2f637564612f637564612d632d70726f6772616d6d696e672d67756964652f67726170686963732f677269642d6f662d7468726561642d626c6f636b732e706e67)](https://camo.githubusercontent.com/87204582bdb0422a08cef47cd0bafcfd860cd522/687474703a2f2f646f63732e6e76696469612e636f6d2f637564612f637564612d632d70726f6772616d6d696e672d67756964652f67726170686963732f677269642d6f662d7468726561642d626c6f636b732e706e67)

We will write a *kernel* that decribes the execution of a single thread in this hierarchy. The CUDA compiler and driver will execute the kernel across a *thread grid* that is divided into *blocks* of threads. Threads within the same block can exchange data very easily during the execution of a kernel, whereas threads in different blocks should generally not communicate with each other (with a few exceptions).

Deciding the best size for the CUDA thread grid is a complex problem (and depends on both the algorithm and the specific GPU compute capability), but here are some very rough heuristics that we follow:

- the size of a block should be a multiple of 32 threads, with typical block sizes between 128 and 512 threads per block.
- the size of the grid should ensure the full GPU is utilized where possible. Launching a grid where the number of blocks is 2x-4x the number of "multiprocessors" on the GPU is a good starting place. Something in the range of 20 - 100 blocks is usually a good starting point.
- The CUDA kernel launch overhead does depend on the number of blocks, so we find it best not to launch a grid where the number of threads equals the number of input elements when the input size is very big. We'll show a pattern for dealing with large inputs below.

Each thread distinguishes itself from the other threads using its unique thread (`threadIdx`) and block (`blockIdx`) index values, which can be multidimensional if launched that way.



### Step 1: Getting Started

In this section, you will learn how to write your own custom CUDA kernels to do accelerated, parallel computing on a GPU, in python with the help of numba and CUDA.


In [ ]:
from numba import cuda

### Step 2: Write a CUDA kernel

CUDA kernels are compiled using the numba.cuda.jit decorator (not to be confused with the numba.jit decorator for the CPU).

In [ ]:
@cuda.jit
def add_kernel(x, y, out):
    tx = cuda.threadIdx.x # this is the unique thread ID within a 1D block
    ty = cuda.blockIdx.x  # Similarly, this is the unique block ID within the 1D grid

    block_size = cuda.blockDim.x  # number of threads per block
    grid_size = cuda.gridDim.x    # number of blocks in the grid
    
    start = tx + ty * block_size
    stride = block_size * grid_size

    # assuming x and y inputs are same length
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]

### Step 3: Call the function


In [ ]:
import numpy as np

n = 100000
x = np.arange(n).astype(np.float32)
y = 2 * x
out = np.empty_like(x)

threads_per_block = 128
blocks_per_grid = 30

add_kernel[blocks_per_grid, threads_per_block](x, y, out)
print(out[:10])

### Step 4: Copy host data to the GPU and return a CUDA device array

In [ ]:
x_device = cuda.to_device(x)
y_device = cuda.to_device(y)
out_device = cuda.device_array_like(x)

In [ ]:
%timeit add_kernel[blocks_per_grid, threads_per_block](x, y, out)

In [ ]:
%timeit add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device); out_device.copy_to_host()

## Kernel Synchronization

CUDA kernel execution is designed to be asynchronous with respect to the host program. This means that the kernel launch (add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device)) returns immediately, allowing the CPU to continue executing while the GPU works in the background. Only host<->device memory copies or an explicit synchronization call will force the CPU to wait until previously queued CUDA kernels are complete.

Always be sure to synchronize with the GPU when benchmarking CUDA kernels!

In [ ]:
# CPU input/output arrays, implied synchronization for memory copies
%time add_kernel[blocks_per_grid, threads_per_block](x, y, out)

In [ ]:
# GPU input/output arrays, no synchronization (but force sync before and after)
cuda.synchronize()
%time add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device)
cuda.synchronize()

In [ ]:
# GPU input/output arrays, include explicit synchronization in timing
cuda.synchronize()
%time add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device); cuda.synchronize()

# **Exercise: Writing CUDA kernels**

Below statement is a pure Python implementation of a function. 

In [ ]:
# Python implementation
def mandelbrot(x, y, max_iters):
  i = 0
  c = complex(x,y)
  z = 0.0j
  for i in range(max_iters):
    z = z * z + c
    if z.real * z.real + z.imag * z.imag >= 4:
        return i
  return 255

Step 1: Modify the example by compiling:
- @jit decorator, to run as native code on the CPU
- @cuda.jit decorator, to run on the GPU

In [ ]:
# Write your own code here

Step 2: Copy host data to the GPU and return a CUDA device array

In [ ]:
# Write your own code here

Step 3: Can you also rewrite the mandel function using @vectorize for the CUDA target?

In [ ]:
# Write your own code here